In [42]:
import pandas as pd

data = pd.read_csv('cardio_dataset-original.csv')

data.head()

,SEX,AGEIR,TC,HDL,SMOKE_,BPMED,DIAB_noyes,RISK
0,female,48,236,66,no smoking,taking,no,1.1
1,male,48,260,51,no smoking,taking,yes,7.0
2,male,44,187,49,smoking,taking,no,7.0
3,female,42,216,57,smoking,taking,no,0.4
4,female,56,156,42,no smoking,taking,no,2.2


# Step 1 - Handling Outliers

In [36]:
factor = 4

upper_lim = data['AGEIR'].mean () + data['AGEIR'].std () * factor
lower_lim = data['AGEIR'].mean () - data['AGEIR'].std () * factor

data_new = data[(data['AGEIR'] < upper_lim) & (data['AGEIR'] > lower_lim)]

In [37]:
upper_lim = data_new['TC'].mean () + data_new['TC'].std () * factor
lower_lim = data_new['TC'].mean () - data_new['TC'].std () * factor

data_new = data_new[(data_new['TC'] < upper_lim) & (data_new['TC'] > lower_lim)]

In [38]:
upper_lim = data_new['HDL'].mean () + data_new['HDL'].std () * factor
lower_lim = data_new['HDL'].mean () - data_new['HDL'].std () * factor

data_new = data_new[(data_new['HDL'] < upper_lim) & (data_new['HDL'] > lower_lim)]

In [39]:
upper_lim = data_new['RISK'].mean () + data_new['RISK'].std () * factor
lower_lim = data_new['RISK'].mean () - data_new['RISK'].std () * factor

data_new = data_new[(data_new['RISK'] < upper_lim) & (data_new['RISK'] > lower_lim)]

# Step 2 - Categorical Encoding

In [40]:
data_new["SEX"] = data_new["SEX"].astype('category')
data_new["SMOKE_"] = data_new["SMOKE_"].astype('category')
data_new["BPMED"] = data_new["BPMED"].astype('category')
data_new["DIAB_noyes"] = data_new["DIAB_noyes"].astype('category')
print(data_new.dtypes)

SEX           category
AGEIR            int64
TC               int64
HDL              int64
SMOKE_        category
BPMED         category
DIAB_noyes    category
RISK           float64
dtype: object


In [41]:
data_new["SEX"] = data_new["SEX"].cat.codes
data_new["SMOKE_"] = data_new["SMOKE_"].cat.codes
data_new["BPMED"] = data_new["BPMED"].cat.codes
data_new["DIAB_noyes"] = data_new["DIAB_noyes"].cat.codes
data_new.head()

,SEX,AGEIR,TC,HDL,SMOKE_,BPMED,DIAB_noyes,RISK
0,0,48,236,66,0,1,0,1.1
1,1,48,260,51,0,1,1,7.0
2,1,44,187,49,1,1,0,7.0
3,0,42,216,57,1,1,0,0.4
4,0,56,156,42,0,1,0,2.2


# Step 3 - Scaling

In [43]:
dataset=data_new.values #numpy

data=dataset[:,:7]
target=dataset[:,7]

print(data)

[[  0.  48. 236. ...   0.   1.   0.]
 [  1.  48. 260. ...   0.   1.   1.]
 [  1.  44. 187. ...   1.   1.   0.]
 ...
 [  1.  65. 212. ...   1.   0.   0.]
 [  1.  66. 184. ...   0.   0.   0.]
 [  0.  45. 203. ...   0.   1.   0.]]


In [29]:
from sklearn.preprocessing import QuantileTransformer
import numpy as np

target=np.reshape(target, (-1,1))

scaler_data = QuantileTransformer()
scaler_target = QuantileTransformer()

data_scaled=scaler_data.fit_transform(data)
target_scaled=scaler_target.fit_transform(target)

# Step 5 - Training

In [30]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data_scaled,target_scaled,test_size=0.1)
print(train_data.shape)

(5908, 7)


In [31]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import numpy as np

model = Sequential()
model.add(Dense(128, input_dim=7, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam',loss='mse',metrics=['mse','mae'])

model.summary()

model.fit(train_data,train_target,epochs=1000,validation_split=0.1)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 128)               1024      
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                650       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 11        
Total params: 9,941
Trainable params: 9,941
Non-trainable params: 0
____________________________________________________

167/167 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - mae: 0.0753 - val_loss: 0.0075 - val_mse: 0.0075 - val_mae: 0.0641
Epoch 93/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0092 - mse: 0.0092 - mae: 0.0730 - val_loss: 0.0075 - val_mse: 0.0075 - val_mae: 0.0629
Epoch 94/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - mae: 0.0726 - val_loss: 0.0075 - val_mse: 0.0075 - val_mae: 0.0647
Epoch 95/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - mae: 0.0738 - val_loss: 0.0075 - val_mse: 0.0075 - val_mae: 0.0644
Epoch 96/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - mae: 0.0724 - val_loss: 0.0075 - val_mse: 0.0075 - val_mae: 0.0626
Epoch 97/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0095 - mse: 0.0095 - mae: 0.0743 - val_loss: 0.0076 - val_mse: 0.0076 - val_mae: 0.0637
Epoch 98/100

Epoch 141/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0084 - mse: 0.0084 - mae: 0.0694 - val_loss: 0.0070 - val_mse: 0.0070 - val_mae: 0.0614
Epoch 142/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0090 - mse: 0.0090 - mae: 0.0724 - val_loss: 0.0073 - val_mse: 0.0073 - val_mae: 0.0617
Epoch 143/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - mae: 0.0716 - val_loss: 0.0072 - val_mse: 0.0072 - val_mae: 0.0617
Epoch 144/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0090 - mse: 0.0090 - mae: 0.0718 - val_loss: 0.0071 - val_mse: 0.0071 - val_mae: 0.0619
Epoch 145/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0085 - mse: 0.0085 - mae: 0.0698 - val_loss: 0.0071 - val_mse: 0.0071 - val_mae: 0.0623
Epoch 146/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0088 - mse: 0.0088 - mae: 0.0714 - val_loss: 0.0070 - val_mse: 0.0070 - val_mae:

Epoch 190/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0081 - mse: 0.0081 - mae: 0.0684 - val_loss: 0.0068 - val_mse: 0.0068 - val_mae: 0.0618
Epoch 191/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0086 - mse: 0.0086 - mae: 0.0702 - val_loss: 0.0067 - val_mse: 0.0067 - val_mae: 0.0600
Epoch 192/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0085 - mse: 0.0085 - mae: 0.0696 - val_loss: 0.0069 - val_mse: 0.0069 - val_mae: 0.0613
Epoch 193/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0081 - mse: 0.0081 - mae: 0.0678 - val_loss: 0.0070 - val_mse: 0.0070 - val_mae: 0.0610
Epoch 194/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0082 - mse: 0.0082 - mae: 0.0687 - val_loss: 0.0068 - val_mse: 0.0068 - val_mae: 0.0607
Epoch 195/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0081 - mse: 0.0081 - mae: 0.0678 - val_loss: 0.0068 - val_mse: 0.0068 - val_mae:

Epoch 239/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0082 - mse: 0.0082 - mae: 0.0684 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae: 0.0603
Epoch 240/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0077 - mse: 0.0077 - mae: 0.0663 - val_loss: 0.0068 - val_mse: 0.0068 - val_mae: 0.0627
Epoch 241/1000
167/167 [==============================] - 0s 2ms/step - loss: 0.0086 - mse: 0.0086 - mae: 0.0696 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae: 0.0603
Epoch 242/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0076 - mse: 0.0076 - mae: 0.0657 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae: 0.0604
Epoch 243/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0079 - mse: 0.0079 - mae: 0.0670 - val_loss: 0.0068 - val_mse: 0.0068 - val_mae: 0.0621
Epoch 244/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0079 - mse: 0.0079 - mae: 0.0663 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae:

...

Epoch 777/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0626 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0580
Epoch 778/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0620 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0593
Epoch 779/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0068 - mse: 0.0068 - mae: 0.0616 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0581
Epoch 780/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0075 - mse: 0.0075 - mae: 0.0643 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0586
Epoch 781/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0074 - mse: 0.0074 - mae: 0.0643 - val_loss: 0.0062 - val_mse: 0.0062 - val_mae: 0.0585
Epoch 782/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0069 - mse: 0.0069 - mae: 0.0620 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae:

Epoch 826/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0074 - mse: 0.0074 - mae: 0.0632 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0590
Epoch 827/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0065 - mse: 0.0065 - mae: 0.0606 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae: 0.0617
Epoch 828/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0629 - val_loss: 0.0067 - val_mse: 0.0067 - val_mae: 0.0631
Epoch 829/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0075 - mse: 0.0075 - mae: 0.0644 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0589
Epoch 830/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0627 - val_loss: 0.0067 - val_mse: 0.0067 - val_mae: 0.0637
Epoch 831/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0635 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae:

Epoch 875/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0625 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0601
Epoch 876/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0073 - mse: 0.0073 - mae: 0.0633 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0604
Epoch 877/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0634 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0592
Epoch 878/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0068 - mse: 0.0068 - mae: 0.0621 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0586
Epoch 879/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0621 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0590
Epoch 880/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0629 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae:

Epoch 924/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0069 - mse: 0.0069 - mae: 0.0613 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae: 0.0617
Epoch 925/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0636 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0593
Epoch 926/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0623 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0599
Epoch 927/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0626 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0594
Epoch 928/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0626 - val_loss: 0.0063 - val_mse: 0.0063 - val_mae: 0.0584
Epoch 929/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0066 - mse: 0.0066 - mae: 0.0602 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae:

Epoch 973/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0070 - mse: 0.0070 - mae: 0.0619 - val_loss: 0.0068 - val_mse: 0.0068 - val_mae: 0.0648
Epoch 974/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0068 - mse: 0.0068 - mae: 0.0617 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae: 0.0633
Epoch 975/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0626 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae: 0.0618
Epoch 976/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0067 - mse: 0.0067 - mae: 0.0607 - val_loss: 0.0065 - val_mse: 0.0065 - val_mae: 0.0619
Epoch 977/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0627 - val_loss: 0.0064 - val_mse: 0.0064 - val_mae: 0.0608
Epoch 978/1000
167/167 [==============================] - 0s 1ms/step - loss: 0.0073 - mse: 0.0073 - mae: 0.0630 - val_loss: 0.0066 - val_mse: 0.0066 - val_mae:

In [32]:
predicted_target=model.predict(test_data)

In [33]:
from sklearn.metrics import r2_score

r2=r2_score(test_target,predicted_target)
print("r2 score:",r2)

r2 score: 0.932388080460625
